# Generate words 2

In [1]:
!pip install torch
!pip install matplotlib
!wget https://gist.githubusercontent.com/craigmartin97/e98a9e2a267c379e47be1191d9431de2/raw/c09c7356e85e39e41faa92a665b7ef0b3b840b6a/last-names.txt

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


7[Files: 0  Bytes: 0  [0 B/s] Re]87[https://gist.githubusercontent]87Saving 'last-names.txt.4'
87last-names.txt.4      28% [=======>                      ]  190.55K    --.-KB/s87[Files: 0  Bytes: 0  [0 B/s] Re]87last-names.txt.4     100% [=============================>]  679.12K    4.41MB/s87HTTP response 200  [https://gist.githubusercontent.com/craigmartin97/e98a9e2a267c379e47be1191d9431de2/raw/c09c7356e85e39e41faa92a665b7ef0b3b840b6a/last-names.txt]
87last-names.txt.4     100% [=============================>]  679.12K    4.41MB/s87[Files: 1  Bytes: 679.12K [610.]8

In [2]:
import numpy as np
import torch
import matplotlib
import matplotlib.pyplot as plt

import torch.nn.functional as F

%matplotlib inline


In [3]:

text = open("last-names.txt").read()
raw_names = text.splitlines()
raw_names = ["." * (blk_size) + name + "." for name in raw_names]
vocab = sorted(list(set(text + ".")))
N = len(vocab)
itc = dict(list(enumerate(vocab)))
cti = {v: k for k, v in itc.items()}


NameError: name 'blk_size' is not defined

### Create the datasets

In [ ]:
X = []
Y = []
blk_size = 3
for name in raw_names:
    for ix in range(len(name) - blk_size):
        x = []
        for jx in range(blk_size):
            x.append(cti[name[ix+jx]])
        X.append(x)
        Y.append(cti[name[ix+blk_size]])

X = torch.tensor(X)
Y = torch.tensor(Y)
Xtr, Ytr = X[:int(len(X) * 0.8)], Y[:int(len(Y) * 0.8)]
Xdev, Ydev = X[int(len(X) * 0.8):int(len(X) * 0.9)], Y[int(len(X) * 0.8):int(len(Y) * 0.9)]
Xval, Yval = X[int(len(X) * 0.9):], Y[int(len(X) * 0.8):]

print(Xtr.shape, Ytr.shape)

### Create the embeddings

`C` is the lookup table to create the embeddings

In [ ]:
emb_dims = 12
C = torch.randn((N, emb_dims))
sample = X[:3]
print(sample)
emb = C[sample]
print(emb)

Now we create the first FC layer, to take the embeddings

In [ ]:
W1 = torch.randn((emb_dims * blk_size, 200)) * 0.1
b1 = torch.randn(200) * 0.1
# here a tanh
W2 = torch.randn((200, N)) * 0.1
b2 = torch.randn(N) * 0.1
# softmax

emb = emb.view(-1, blk_size * emb_dims)
emb.shape

This is how you would do a single forward pass

In [ ]:
a = emb @ W1 + b1
a = F.tanh(a)
a = a @ W2 + b2
print(a.shape)
a = F.softmax(a, 1)
a.shape

In [ ]:
parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

Lets do the training loop

In [ ]:
for p in parameters:
    p.grad = None

In [ ]:
print(len(Xtr))
loss_t = 0
ix = 1
for i in range(200000):
    batch_ix = torch.randint(0, len(Xtr), (32,))

    emb = C[Xtr[batch_ix]]
    emb = emb.view(-1, blk_size * emb_dims)

    h = F.tanh(emb @ W1 + b1)
    logits = h @ W2 + b2
    # probs = F.softmax(logits, 1)
    loss = F.cross_entropy(logits, Ytr[batch_ix])
    lr = 0.01 if i < 100000 else 0.01
    for p in parameters:
        p.grad = None
    
    loss.backward()
    loss_t += loss.item()
    for p in parameters:
        p.data += -lr * p.grad
    if ix % 20000 == 0:
        print(f"{loss_t/ix=}")

    ix += 1
 

In [ ]:
#plt.hist(logits.detach().numpy())

plt.hist(h.detach().numpy())
# plt.imshow(h.detach().numpy() > 0.99)

Now let's use the dev dataset to check our results and tune our hyperparameters

In [ ]:
with torch.no_grad():
    emb = C[Xdev]
    emb = emb.view(-1, blk_size * emb_dims)
    
    h = F.tanh(emb @ W1 + b1)
    logits = h @ W2 + b2
    # probs = F.softmax(logits, 1)
    loss = F.cross_entropy(logits, Ydev)
    print(f"{loss=}")

Lets generate some words

In [ ]:
with torch.no_grad():
    sample = torch.tensor([1, 1,  1])
    out = ""
    while True:
        emb = C[sample]
        emb = emb.view(-1, blk_size * emb_dims)
        
        h = F.tanh(emb @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, 1)
        y = torch.multinomial(probs, num_samples=1, replacement=True)
        sample = sample[1:]
        sample = torch.cat((sample, torch.tensor([y])), 0)
        
        if y == 1:
            break
        out += itc[y.item()]
        
    print(out)

## Some names generated my the model
* WALFAILMAN
* TATTIN
* DUS
* SETTE
* BURG
* MAGNIS
* GORD
* HURN
* QUISGE